In [1]:
import numpy as np
import sympy as sp
from get_metrices import get_Q_matrix, get_T_matrix, get_R_IB

# Clear all symbolic variables
sp.init_printing()

# Define symbols
phi, theta, psi, X, Y, Z, n1, n2, n3 = sp.symbols('phi theta psi X Y Z n1 n2 n3', real=True)
phiDot, thetaDot, psiDot, XDot, YDot, ZDot, n1Dot, n2Dot, n3Dot = sp.symbols('phiDot thetaDot psiDot XDot YDot ZDot n1Dot n2Dot n3Dot', real=True)
d0, d1, d2, d3 = sp.symbols('d0 d1 d2 d3', real=True)

# Position and velocity vectors
P = sp.Matrix([X, Y, Z])
PDot = sp.Matrix([XDot, YDot, ZDot])
PHI = sp.Matrix([phi, theta, psi])
PHIDot = sp.Matrix([phiDot, thetaDot, psiDot])
N = sp.Matrix([n1, n2, n3])
NDot = sp.Matrix([n1Dot, n2Dot, n3Dot])

q = sp.Matrix.vstack(P, PHI, N)
qDot = sp.Matrix.vstack(PDot, PHIDot, NDot)

# Rotation matrices
R_IB = get_R_IB(phi, theta, psi , symbolic=True) # rotation matrix round XYZ from body to inersia frame
Q = get_Q_matrix(phi, theta, psi, symbolic=True) # wb_B = (p, q, r)' = Q*(phiDot, thetaDot, psiDot)'
T = get_T_matrix(phi, theta, psi, symbolic=True) # wb_I = (wx,wy,wz)' = R_IB*wb_B = R_IB*(p, q, r)' 
# T = sp.simplify(R_IB * Q)

print(T, "shape: ", T.shape)
print(Q, "shape: ", Q.shape)


Matrix([[cos(psi)*cos(theta), -sin(psi), 0], [sin(psi)*cos(theta), cos(psi), 0], [-sin(theta), 0, 1]]) shape:  (3, 3)
Matrix([[1, 0, -sin(theta)], [0, cos(phi), sin(phi)*cos(theta)], [0, -sin(phi), cos(phi)*cos(theta)]]) shape:  (3, 3)


In [2]:
# # Define the angular velocity vector in the body frame
# angular_velocity_body = Rz * Ry * sp.Matrix([phiDot, 0, 0]) + Rz * sp.Matrix([0, thetaDot, 0]) + sp.Matrix([0, 0, psiDot])

# T = sp.simplify(sp.Matrix([angular_velocity_body]).jacobian(PHIDot))
# Q = sp.simplify(R_IB.T * T)

# print("T:", T, "shape: ", T.shape)
# print("Q:", Q, "shape: ", Q.shape)

In [3]:

def HTM(theta, d, a, alpha):
    """
    Homogeneous Transformation Matrix (HTM)

    Args:
        theta: Rotation angle about z-axis (in radians).
        d: Translation along z-axis.
        a: Translation along x-axis.
        alpha: Rotation angle about x-axis (in radians).

    Returns:
        H_: Homogeneous Transformation Matrix
    """

    # Rotation about z-axis (in radians)
    Rot_z = sp.Matrix([
        [sp.cos(theta), -sp.sin(theta), 0, 0],
        [sp.sin(theta),  sp.cos(theta), 0, 0],
        [0,              0,             1, 0],
        [0,              0,             0, 1]
    ])

    # Translation along z-axis
    Trans_z = sp.Matrix([
        [1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 1, d],
        [0, 0, 0, 1]
    ])

    # Translation along x-axis
    Trans_x = sp.Matrix([
        [1, 0, 0, a],
        [0, 1, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]
    ])

    # Rotation about x-axis (in radians)
    Rot_x = sp.Matrix([
        [1, 0,           0,            0],
        [0, sp.cos(alpha), -sp.sin(alpha), 0],
        [0, sp.sin(alpha),  sp.cos(alpha), 0],
        [0, 0,           0,            1]
    ])

    # Homogeneous Transformation Matrix
    H_ = Rot_z * Trans_z * Trans_x * Rot_x

    return sp.simplify(H_)

# Example usage (if needed):
theta, d, a, alpha = sp.symbols('theta d a alpha')
H_matrix = HTM(theta, d, a, alpha)
print("Homogeneous Transformation Matrix (HTM):")
sp.pprint(H_matrix)


Homogeneous Transformation Matrix (HTM):
⎡cos(θ)  -sin(θ)⋅cos(α)  sin(α)⋅sin(θ)   a⋅cos(θ)⎤
⎢                                                ⎥
⎢sin(θ)  cos(α)⋅cos(θ)   -sin(α)⋅cos(θ)  a⋅sin(θ)⎥
⎢                                                ⎥
⎢  0         sin(α)          cos(α)         d    ⎥
⎢                                                ⎥
⎣  0           0               0            1    ⎦


In [4]:
def H2RP(H):
    """
    Extract the rotation matrix and position vector from a Homogeneous Transformation Matrix.
    
    Parameters:
    H (numpy.ndarray): A 4x4 homogeneous transformation matrix.
    
    Returns:
    tuple:
        - R (numpy.ndarray): A 3x3 rotation matrix.
        - P (numpy.ndarray): A 3x1 position vector.
    """
    # Ensure H is a 4x4 matrix
    H = np.reshape(H, (4, 4))
    
    # Extract the rotation matrix (upper-left 3x3 submatrix)
    R = H[:3, :3]
    
    # Extract the position vector (first three entries of the fourth column)
    P = H[:3, 3]
    
    return sp.Matrix(R), sp.Matrix(P)


In [5]:
def Jacobian_w(R_b0, R_b1, R_b2, alpha):
    """
    Calculate the Jacobian of Angular velocity of the center of mass (COM).
    
    Parameters:
    R_b0, R_b1, R_b2 : sympy.Matrix
        Rotation matrices for each link.
    alpha : list or sympy.Matrix
        A vector of joint types (1 for revolute, 0 for prismatic).
        
    Returns:
    Jw : sympy.Matrix
        The angular velocity Jacobian.
    """
    num_alpha = len(alpha)
    
    # Define the e3 vector (unit vector along z-axis)
    e3 = sp.Matrix([0, 0, 1])
    
    # Initialize the Jacobian matrix Jw
    Jw = sp.Matrix.zeros(3, num_alpha)

    Jw[0:3, 0] = alpha[0] * R_b0 * e3
    Jw[0:3, 1] = alpha[1] * R_b1 * e3
    Jw[0:3, 2] = alpha[2] * R_b2 * e3

    return sp.Matrix(Jw)


In [6]:
def skew_symmetric(vector):
    """
    Create a skew-symmetric matrix for a given 3D vector.
    
    Parameters:
    vector : sympy.Matrix
        A 3D vector to create the skew-symmetric matrix.
        
    Returns:
    skew_matrix : sympy.Matrix
        The corresponding skew-symmetric matrix.
    """
    vector = vector.reshape(3,1)
    
    return sp.Matrix([[0, -vector[2], vector[1]],
                   [vector[2], 0, -vector[0]],
                   [-vector[1], vector[0], 0]])

In [7]:
# Homogeneous Transformation Matrices (HTM)
# def HTM(theta, d, a, alpha ):
#     return sp.Matrix([
#         [sp.cos(theta), -sp.sin(theta) * sp.cos(alpha), sp.sin(theta) * sp.sin(alpha), a * sp.cos(theta)],
#         [sp.sin(theta), sp.cos(theta) * sp.cos(alpha), -sp.cos(theta) * sp.sin(alpha), a * sp.sin(theta)],
#         [0, sp.sin(alpha), sp.cos(alpha), d],
#         [0, 0, 0, 1]
#     ])

H_b0 = HTM(0, d0, 0, sp.pi / 2)
H_01 = HTM(n1, 0, d1, 0)
H_12 = HTM(n2, 0, d2, 0)
H_2e = HTM(n3, 0, d3, 0)

pl0_b = sp.Matrix([0, 0, d0 / 2])  # COM of Link0 in the body frame

R_b0, p_b0 = H2RP(H_b0)
pl1_b = sp.simplify(p_b0 + R_b0 @ sp.Matrix([d1 / 2 * sp.cos(n1), d1 / 2 * sp.sin(n1), 0]))  # COM of Link1 in the body frame

H_b1 = H_b0 * H_01
R_b1, p_b1 = H2RP(H_b1)
pl2_b = sp.simplify(p_b1 + R_b1 @ sp.Matrix([d2 / 2 * sp.cos(n2), d2 / 2 * sp.sin(n2), 0]))  # COM of Link2 in the body frame

H_b2 = H_b1 * H_12
R_b2, p_b2 = H2RP(H_b2)
pl3_b = sp.simplify(p_b2 + R_b2 @ sp.Matrix([d3 / 2 * sp.cos(n3), d3 / 2 * sp.sin(n3), 0]))  # COM of Link3 in the body frame

H_be = H_b2 * H_2e
R_be, p_be = H2RP(H_be)


# print("COM of Link0 in body frame (pl0_b):")
# print(pl0_b)
# print("\nCOM of Link1 in body frame (pl1_b):")
# print(pl1_b)
# print("\nCOM of Link2 in body frame (pl2_b):")
# print(pl2_b)
# print("\nCOM of Link3 in body frame (pl3_b):")
# print(pl3_b)


In [8]:
# Compute Jacobians for COMs using the `.jacobian` method
J_t0 = sp.simplify(pl0_b.jacobian(sp.Matrix([n1, n2, n3])))  # Jacobian for Link0
J_t1 = sp.simplify(pl1_b.jacobian(sp.Matrix([n1, n2, n3])))  # Jacobian for Link1
J_t2 = sp.simplify(pl2_b.jacobian(sp.Matrix([n1, n2, n3])))  # Jacobian for Link2
J_t3 = sp.simplify(pl3_b.jacobian(sp.Matrix([n1, n2, n3])))  # Jacobian for Link3


# # Outputs
# print("Jacobian of COM for Link0 (J_t0):")
# print(J_t0)
# print("\nJacobian of COM for Link1 (J_t1):")
# print(J_t1)
# print("\nJacobian of COM for Link2 (J_t2):")
# print(J_t2)
# print("\nJacobian of COM for Link3 (J_t3):")
# print(J_t3)

In [9]:
# rotational Jacobian of Links
J_r0 = Jacobian_w(R_b0, R_b1, R_b2, sp.Matrix([0,0,0]))
J_r1 = Jacobian_w(R_b0, R_b1, R_b2, sp.Matrix([1,0,0]))
J_r2 = Jacobian_w(R_b0, R_b1, R_b2, sp.Matrix([1,1,0]))
J_r3 = Jacobian_w(R_b0, R_b1, R_b2, sp.Matrix([1,1,1]))

# # Outputs
# print("Jacobian of Angular Velocity (J_r0):")
# print(J_r0)
# print("\nJacobian of Angular Velocity (J_r1):")
# print(J_r1)
# print("\nJacobian of Angular Velocity (J_r2):")
# print(J_r2)
# print("\nJacobian of Angular Velocity (J_r3):")
# print(J_r3)


In [10]:
# Define the matrices M_tb and M_rb
M_tb = sp.Matrix.hstack(sp.eye(3), sp.zeros(3, 3), sp.zeros(3, 3))
M_rb = sp.Matrix.hstack(sp.zeros(3, 3), T, sp.zeros(3, 3))

# # M_t0, M_t1, M_t2, M_t3: Define the matrices for translational components
M_t0 = sp.Matrix.hstack(sp.eye(3), -skew_symmetric(R_IB * pl0_b) * T, R_IB * J_t0)
M_t1 = sp.Matrix.hstack(sp.eye(3), -skew_symmetric(R_IB * pl1_b) * T, R_IB * J_t1)
M_t2 = sp.Matrix.hstack(sp.eye(3), -skew_symmetric(R_IB * pl2_b) * T, R_IB * J_t2)
M_t3 = sp.Matrix.hstack(sp.eye(3), -skew_symmetric(R_IB * pl3_b) * T, R_IB * J_t3)

# # M_r0, M_r1, M_r2, M_r3: Define the matrices for rotational components
M_r0 = sp.Matrix.hstack(sp.zeros(3, 3), T, R_IB * J_r0)
M_r1 = sp.Matrix.hstack(sp.zeros(3, 3), T, R_IB * J_r1)
M_r2 = sp.Matrix.hstack(sp.zeros(3, 3), T, R_IB * J_r2)
M_r3 = sp.Matrix.hstack(sp.zeros(3, 3), T, R_IB * J_r3)

# Print the results for verification
# print("M_tb:", M_tb)
# print("M_rb:", M_rb)
# print("M_t0:", M_t0)
# print("M_t1:", M_t1)
# print("M_t2:", M_t2)
# print("M_t3:", M_t3)
# print("M_r0:", M_r0)
# print("M_r1:", M_r1)
# print("M_r2:", M_r2)
# print("M_r3:", M_r3)

In [11]:
PDot = sp.simplify(M_tb*qDot)
omga = sp.simplify(M_rb*qDot)

Pl0Dot = sp.simplify(M_t0*qDot)
Pl1Dot = sp.simplify(M_t1*qDot)
Pl2Dot = sp.simplify(M_t2*qDot)
Pl3Dot = sp.simplify(M_t3*qDot)

omga0 = sp.simplify(M_r0*qDot)
omga1 = sp.simplify(M_r1*qDot)
omga2 = sp.simplify(M_r2*qDot)
omga3 = sp.simplify(M_r3*qDot)

In [12]:
# Define symbolic variables for mass, lengths, moments of inertia, and gravity
mb, mL0, mL1, mL2, mL3 = sp.symbols('mb mL0 mL1 mL2 mL3', real=True)  # masses
Ixx, Iyy, Izz, Ix0, Ix1, Ix2, Ix3, Iy0, Iy1, Iy2, Iy3, Iz0, Iz1, Iz2, Iz3 = \
    sp.symbols('Ixx Iyy Izz Ix0 Ix1 Ix2 Ix3 Iy0 Iy1 Iy2 Iy3 Iz0 Iz1 Iz2 Iz3', real=True)  # moments of inertia
g = sp.symbols('g', real=True)  # gravitational constant

# Create the inertia matrices using diag
Ib = sp.Matrix.diag(Ixx, Iyy, Izz)  # Inertia matrix for the body
IL0 = sp.Matrix.diag(Ix0, Iy0, Iz0)  # Inertia matrix for Link 0
IL1 = sp.Matrix.diag(Ix1, Iy1, Iz1)  # Inertia matrix for Link 1
IL2 = sp.Matrix.diag(Ix2, Iy2, Iz2)  # Inertia matrix for Link 2
IL3 = sp.Matrix.diag(Ix3, Iy3, Iz3)  # Inertia matrix for Link 3


In [13]:
# Kinetic energy terms for body (Kb), and links (K0, K1, K2, K3)

Kb = 0.5*mb*(PDot.T*PDot) + 0.5*PHI.T*T.T*R_IB*Ib*R_IB.T*T*PHIDot
K0 = 0.5*mL0*(Pl0Dot.T*Pl0Dot) + 0.5*omga0.T*R_IB*IL0*R_IB.T*omga0
K1 = 0.5*mL1*(Pl1Dot.T*Pl1Dot) + 0.5*omga1.T*(R_IB*R_b0)*IL1*(R_IB*R_b0).T*omga1
K2 = 0.5*mL2*(Pl2Dot.T*Pl2Dot) + 0.5*omga2.T*(R_IB*R_b1)*IL2*(R_IB*R_b1).T*omga2
K3 = 0.5*mL3*(Pl3Dot.T*Pl3Dot) + 0.5*omga3.T*(R_IB*R_b2)*IL3*(R_IB*R_b2).T*omga3

# Total kinetic energy
# K = sp.simplify(Kb + K0 + K1 + K2 + K3)
K = Kb + K0 + K1 + K2 + K3

e3 = sp.Matrix([0, 0, 1])  # Unit vector in z direction

# Gravitational potential energy (U)
U = -sp.simplify(
    mb * g * e3.T * P +
    mL0 * g * e3.T * (P + R_IB * pl0_b) +
    mL1 * g * e3.T * (P + R_IB * pl1_b) +
    mL2 * g * e3.T * (P + R_IB * pl2_b) +
    mL3 * g * e3.T * (P + R_IB * pl3_b)
)

# Lagrangian (L) = Kinetic Energy (K) - Potential Energy (U)
L = K - U

In [14]:

# Generalized forces (Gq) - Jacobian of U with respect to generalized coordinates q
Gq = (U.jacobian(q)).T

# mass/inertia matrix
Mq = \
    M_tb.T * mb * M_tb + M_rb.T * R_IB * Ib * R_IB.T * M_rb + \
    M_t0.T * mL0 * M_t0 + M_r0.T * R_IB * IL0 * R_IB.T * M_r0 + \
    M_t1.T * mL1 * M_t1 + M_r1.T * (R_IB * R_b0) * IL1 * (R_IB * R_b0).T * M_r1 + \
    M_t2.T * mL2 * M_t2 + M_r2.T * (R_IB * R_b1) * IL2 * (R_IB * R_b1).T * M_r2 + \
    M_t3.T * mL3 * M_t3 + M_r3.T * (R_IB * R_b2) * IL3 * (R_IB * R_b2).T * M_r3


n_q = len(q)  # Number of generalized coordinates
Cq = sp.zeros(n_q) # Initialize Coriolis matrix Cq
# Calculate the Coriolis matrix
for k in range(n_q):
    for j in range(n_q):
        for i in range(n_q):
            m_kj = Mq[k, j]
            m_ki = Mq[k, i]
            m_ij = Mq[i, j]
            
            # Compute the Coriolis term for each entry
            Cq[k, j] += 0.5 * (sp.diff(m_kj, q[i]) + sp.diff(m_ki, q[j]) - sp.diff(m_ij, q[k])) * qDot[i]




In [15]:
# # Simplify component-wise with trigsimp
Gq = sp.simplify(Gq)

subexprs_Gq, simplified_Gq = sp.cse(Gq)
simplified_Gq = np.array(simplified_Gq).reshape(Gq.shape)


with open("get_Gq.py", "w") as f:
    f.write("import numpy as np\n")
    f.write("from math import * \n\n")
    f.write("def get_Gq(q, qDot, params):\n\n")

    f.write(f"\tg, mb, mL0, mL1, mL2, mL3, d, d0, d1, d2, d3 = \\\n\t\tparams['g'], params['mb'], params['mL0'], params['mL1'], params['mL2'], params['mL3'], params['d'], params['d0'], params['d1'], params['d2'], params['d3']\n")
    
    f.write(f"\tIxx, Iyy, Izz, Ix0, Iy0, Iz0, Ix1, Iy1, Iz1, Ix2, Iy2, Iz2, Ix3, Iy3, Iz3 = \\\n\t\tparams['Ixx'], params['Iyy'], params['Izz'], params['Ix0'], params['Iy0'], params['Iz0'], params['Ix1'], params['Iy1'], params['Iz1'], params['Ix2'], params['Iy2'], params['Iz2'], params['Ix3'], params['Iy3'], params['Iz3']\n")

    f.write(f"\tX, Y, Z, phi, theta, psi, n1, n2, n3 = \\\n\t\tq[0], q[1], q[2], q[3], q[4], q[5], q[6], q[7], q[8] \n")
    
    f.write(f"\tXDot, YDot, ZDot, phiDot, thetaDot, psiDot, n1Dot, n2Dot, n3Dot = \\\n\t\tqDot[0], qDot[1], qDot[2], qDot[3], qDot[4], qDot[5], qDot[6], qDot[7], qDot[8] \n")

    f.write(f"\tGq = np.zeros(({simplified_Gq.shape[0]},{simplified_Gq.shape[1]})) \n\n")
    for sub in subexprs_Gq:
        f.write(f"\t{sub[0]} = {sub[1]}\n")
    f.write("\n")
    
    for i in range(simplified_Gq.shape[0]) :
        for j in range(simplified_Gq.shape[1]):
            f.write(f"\tGq[{i},{j}] = {simplified_Gq[i,j]} \n")
            
    f.write(f"\n\n")
    f.write(f"\treturn Gq")
    f.close()

In [16]:

subexprs_Mq, simplified_Mq = sp.cse(Mq)
simplified_Mq = np.array(simplified_Mq).reshape(Mq.shape)

with open("get_Mq.py", "w") as f:
    f.write("import numpy as np\n")
    f.write("from math import * \n\n")
    f.write("def get_Mq(q, qDot, params):\n\n")

    f.write(f"\tg, mb, mL0, mL1, mL2, mL3, d, d0, d1, d2, d3 = \\\n\t\tparams['g'], params['mb'], params['mL0'], params['mL1'], params['mL2'], params['mL3'], params['d'], params['d0'], params['d1'], params['d2'], params['d3']\n")
    
    f.write(f"\tIxx, Iyy, Izz, Ix0, Iy0, Iz0, Ix1, Iy1, Iz1, Ix2, Iy2, Iz2, Ix3, Iy3, Iz3 = \\\n\t\tparams['Ixx'], params['Iyy'], params['Izz'], params['Ix0'], params['Iy0'], params['Iz0'], params['Ix1'], params['Iy1'], params['Iz1'], params['Ix2'], params['Iy2'], params['Iz2'], params['Ix3'], params['Iy3'], params['Iz3']\n")

    f.write(f"\tX, Y, Z, phi, theta, psi, n1, n2, n3 = \\\n\t\tq[0], q[1], q[2], q[3], q[4], q[5], q[6], q[7], q[8] \n")
    
    f.write(f"\tXDot, YDot, ZDot, phiDot, thetaDot, psiDot, n1Dot, n2Dot, n3Dot = \\\n\t\tqDot[0], qDot[1], qDot[2], qDot[3], qDot[4], qDot[5], qDot[6], qDot[7], qDot[8] \n")

    f.write(f"\tMq = np.zeros(({simplified_Mq.shape[0]},{simplified_Mq.shape[1]})) \n\n")
    for sub in subexprs_Mq:
        f.write(f"\t{sub[0]} = {sub[1]}\n")
    f.write("\n")
    
    for i in range(simplified_Mq.shape[0]) :
        for j in range(simplified_Mq.shape[1]):
            f.write(f"\tMq[{i},{j}] = {simplified_Mq[i,j]} \n")
            
    f.write(f"\n\n")
    f.write(f"\treturn Mq")
    f.close()

In [17]:

subexprs_Cq, simplified_Cq = sp.cse(Cq)
simplified_Cq = np.array(simplified_Cq).reshape(Cq.shape)

with open("get_Cq.py", "w") as f:
    f.write("import numpy as np\n")
    f.write("from math import * \n\n")
    f.write("def get_Cq(q, qDot, params):\n\n")

    f.write(f"\tg, mb, mL0, mL1, mL2, mL3, d, d0, d1, d2, d3 = \\\n\t\tparams['g'], params['mb'], params['mL0'], params['mL1'], params['mL2'], params['mL3'], params['d'], params['d0'], params['d1'], params['d2'], params['d3']\n")
    
    f.write(f"\tIxx, Iyy, Izz, Ix0, Iy0, Iz0, Ix1, Iy1, Iz1, Ix2, Iy2, Iz2, Ix3, Iy3, Iz3 = \\\n\t\tparams['Ixx'], params['Iyy'], params['Izz'], params['Ix0'], params['Iy0'], params['Iz0'], params['Ix1'], params['Iy1'], params['Iz1'], params['Ix2'], params['Iy2'], params['Iz2'], params['Ix3'], params['Iy3'], params['Iz3']\n")

    f.write(f"\tX, Y, Z, phi, theta, psi, n1, n2, n3 = \\\n\t\tq[0], q[1], q[2], q[3], q[4], q[5], q[6], q[7], q[8] \n")
    
    f.write(f"\tXDot, YDot, ZDot, phiDot, thetaDot, psiDot, n1Dot, n2Dot, n3Dot = \\\n\t\tqDot[0], qDot[1], qDot[2], qDot[3], qDot[4], qDot[5], qDot[6], qDot[7], qDot[8] \n")

    f.write(f"\tCq = np.zeros(({simplified_Cq.shape[0]},{simplified_Cq.shape[1]})) \n\n")
    for sub in subexprs_Cq:
        f.write(f"\t{sub[0]} = {sub[1]}\n")
    f.write("\n")
    
    for i in range(simplified_Cq.shape[0]) :
        for j in range(simplified_Cq.shape[1]):
            f.write(f"\tCq[{i},{j}] = {simplified_Cq[i,j]} \n")
            
    f.write(f"\n\n")
    f.write(f"\treturn Cq")
    f.close()